In [ ]:
#Part 1: Create Country HWP Dataframe from FAOSTAT data 

import pandas a
import numpy as np

def load_country_codes(file_path):
    """
    Load country codes from the FAO area codes CSV file
    """
    try:
        country_df = pd.read_csv(file_path)

        area_col = next((col for col in country_df.columns if 'area' in col.lower() and not 'code' in col.lower()), None)
        code_col = next((col for col in country_df.columns if 'code' in col.lower() and 'area' in col.lower()), None)

        if area_col is None or code_col is None:
            raise ValueError(f"Could not find required columns. Available columns: {country_df.columns.tolist()}")

        country_dict = dict(zip(country_df[area_col].str.lower(), country_df[code_col]))
        return country_df, country_dict
    except Exception as e:
        print(f"Error loading country codes: {e}")
        return None, None

def get_country_code(country_dict, country_name):
    """
    Get the area code for a given country name
    """
    country_name_lower = country_name.lower()
    return country_dict.get(country_name_lower, None)

def process_country_data(fao_df, area_code):
    """
    Process the FAO data for the specified country and return a formatted DataFrame
    """
    product_codes = {
        '1865': 'industrial_roundwood',
        '1876': 'paper',
        '1872': 'sawnwood',
        '1875': 'woodpulp',
        '1873': 'woodpanels'
    }

    headers = [
        'Area', 'year',
        'industrial_roundwood_export', 'industrial_roundwood_import', 'industrial_roundwood_production',
        'paper_export', 'paper_import', 'paper_production',
        'sawnwood_export', 'sawnwood_import', 'sawnwood_production',
        'woodpulp_export', 'woodpulp_import', 'woodpulp_production',
        'woodpanels_export', 'woodpanels_import', 'woodpanels_production'
    ]

    data = []
    country_data = fao_df[fao_df['Area Code'].astype(str) == str(area_code)]

    for year in sorted(country_data['Year'].unique()):
        row = {'Area': country_data['Area'].iloc[0], 'year': year}
        for code, product in product_codes.items():
            product_data = country_data[country_data['Item Code'].astype(str) == code]
            row[f'{product}_export'] = product_data[(product_data['Element'] == 'Export quantity') & (product_data['Year'] == year)]['Value'].sum()
            row[f'{product}_import'] = product_data[(product_data['Element'] == 'Import quantity') & (product_data['Year'] == year)]['Value'].sum()
            row[f'{product}_production'] = product_data[(product_data['Element'] == 'Production') & (product_data['Year'] == year)]['Value'].sum()
        data.append(row)

    return pd.DataFrame(data, columns=headers)

def main():
    print("Loading country codes...")
    country_df, country_dict = load_country_codes('Forestry_E_AreaCodes.csv')

    if country_dict is None:
        print("Failed to load country codes. Exiting.")
        return

    while True:
        country_name = input("\nEnter country name (e.g., Austria): ")
        area_code = get_country_code(country_dict, country_name)

        if area_code is None:
            print(f"Country '{country_name}' not found. Please try again.")
            show_all = input("Would you like to see all available countries? (y/n): ").lower()
            if show_all == 'y':
                print("\nAvailable countries:")
                for country in sorted(country_dict.keys()):
                    print(country.title())
        else:
            country_name = next(name.title() for name, code in country_dict.items() if code == area_code)
            print(f"\nSelected: {country_name} (Area Code: {area_code})")
            break

    try:
        fao_df = pd.read_csv('Forestry_E_All_Data_(Normalized).csv')
    except Exception as e:
        print(f"Error loading FAO data: {e}")
        return

    try:
        result_df = process_country_data(fao_df, area_code)
        print("\nGenerated DataFrame:")
        print(result_df.head())
        # Save to a CSV if needed
        result_df.to_csv(f'{country_name}_processed_data.csv', index=False)
        print(f"Data saved to {country_name}_processed_data.csv")
    except Exception as e:
        print(f"\nUnexpected error: {e}")

if __name__ == "__main__":
    main()


In [ ]:
## Part 2 : Convert FAO Data to HWP Carbon Stock Change Estimates

import pandas as pd
import numpy as np

# Read the CSV file
df = pd.read_csv('Austria_processed_data.csv')
print("Data loaded. Shape:", df.shape)
print("\nFirst few rows:")
print(df.head())

print("\nChecking for missing values:")
print(df.isnull().sum())

print("\nColumn data types:")
print(df.dtypes)

# User input for calculation options
while True:
    include_domestic = input("Do you want to include domestic fraction calculations? (yes/no): ").lower()
    if include_domestic in ['yes', 'no']:
        break
    print("Please enter either 'yes' or 'no'")

while True:
    initial_stock_method = input("How do you want to calculate initial carbon stocks? (zero/formula): ").lower()
    if initial_stock_method in ['zero', 'formula']:
        break
    print("Please enter either 'zero' or 'formula'")

while True:
    inflow_timing = input("Which inflow timing do you want to use for stock calculation? (current/previous): ").lower()
    if inflow_timing in ['current', 'previous']:
        break
    print("Please enter either 'current' or 'previous'")

# New prompt for initial year
while True:
    try:
        initial_year = int(input("Enter the initial year for calculations (e.g., 1900): "))
        if 1900 <= initial_year <= 1961:
            break
        print("Please enter a year between 1900 and 1961")
    except ValueError:
        print("Please enter a valid year (integer)")

# Ensure 'year' is integer type
df['year'] = df['year'].astype(int)

# Carbon conversion factors
cf_sawnwood = 0.229
cf_woodpanels = 0.269
cf_paper = 0.386

# Half-lives (in years)
hl_sawnwood = 35
hl_woodpanels = 25
hl_paper = 2

# Calculate decay constants (k)
k_sawnwood = np.log(2) / hl_sawnwood
k_woodpanels = np.log(2) / hl_woodpanels
k_paper = np.log(2) / hl_paper

# Function to calculate domestic fraction
def calculate_industrial_domestic_fraction(production, import_quantity, export_quantity):
    return np.where(
        production > 0,
        np.maximum(0, np.minimum(1, (production - export_quantity) / (production + import_quantity - export_quantity))),
        0
    )

try:
    if include_domestic == 'yes':
        # Calculate fractions for each product
        df['industrial_roundwood_fraction'] = calculate_industrial_domestic_fraction(
            df['industrial_roundwood_production'], 
            df['industrial_roundwood_import'], 
            df['industrial_roundwood_export']
        )
        df['paper_fraction'] = calculate_industrial_domestic_fraction(
            df['woodpulp_production'], 
            df['woodpulp_import'], 
            df['woodpulp_export']
        )

        # Calculate domestic production volumes
        df['sawnwood_domestic_production'] = df['sawnwood_production'] * df['industrial_roundwood_fraction']
        df['woodpanels_domestic_production'] = df['woodpanels_production'] * df['industrial_roundwood_fraction']
        df['paper_domestic_production'] = df['paper_production'] * df['industrial_roundwood_fraction'] * df['paper_fraction']
        
        # Calculate export production
        df['sawnwood_export_production'] = df['sawnwood_production'] - df['sawnwood_domestic_production']
        df['woodpanels_export_production'] = df['woodpanels_production'] - df['woodpanels_domestic_production']
        df['paper_export_production'] = df['paper_production'] - df['paper_domestic_production']
    
    else:
        # Use total production without domestic fraction
        df['sawnwood_domestic_production'] = df['sawnwood_production'] - df['sawnwood_export']
        df['woodpanels_domestic_production'] = df['woodpanels_production'] - df['woodpanels_export']
        df['paper_domestic_production'] = df['paper_production'] - df['paper_export']
        
        df['sawnwood_export_production'] = df['sawnwood_export']
        df['woodpanels_export_production'] = df['woodpanels_export']
        df['paper_export_production'] = df['paper_export']

    # Calculate inflows
    # Domestic inflows
    df['sawnwood_domestic_inflow'] = df['sawnwood_domestic_production'] * cf_sawnwood
    df['woodpanels_domestic_inflow'] = df['woodpanels_domestic_production'] * cf_woodpanels
    df['paper_domestic_inflow'] = df['paper_domestic_production'] * cf_paper
    
    # Export inflows
    df['sawnwood_export_inflow'] = df['sawnwood_export_production'] * cf_sawnwood
    df['woodpanels_export_inflow'] = df['woodpanels_export_production'] * cf_woodpanels
    df['paper_export_inflow'] = df['paper_export_production'] * cf_paper

    # Function to estimate pre-1961 inflow
    def estimate_pre1961_inflow(Vr, U, t, t0):
        return Vr * np.exp(U * (t - t0))

    # Constants for pre-1961 estimates
    t0 = 1961
    U = 0.0151

    # Calculate Vr 1961 for domestic and export separately
    Vr_domestic = {
        'sawnwood': df['sawnwood_domestic_production'].iloc[0] * cf_sawnwood,
        'woodpanels': df['woodpanels_domestic_production'].iloc[0] * cf_woodpanels,
        'paper': df['paper_domestic_production'].iloc[0] * cf_paper
    }
    
    Vr_export = {
        'sawnwood': df['sawnwood_export_production'].iloc[0] * cf_sawnwood,
        'woodpanels': df['woodpanels_export_production'].iloc[0] * cf_woodpanels,
        'paper': df['paper_export_production'].iloc[0] * cf_paper
    }

    # Estimate inflows for years prior to 1961, starting from user-specified initial year
    pre1961_inflows = {
        'domestic': {product: [] for product in ['sawnwood', 'woodpanels', 'paper']},
        'export': {product: [] for product in ['sawnwood', 'woodpanels', 'paper']}
    }
    pre1961_years = list(range(initial_year, t0-1))

    for year in pre1961_years:
        for product in ['sawnwood', 'woodpanels', 'paper']:
            domestic_inflow = estimate_pre1961_inflow(Vr_domestic[product], U, year, t0)
            export_inflow = estimate_pre1961_inflow(Vr_export[product], U, year, t0)
            
            pre1961_inflows['domestic'][product].append(domestic_inflow)
            pre1961_inflows['export'][product].append(export_inflow)

    # Create pre-1961 dataframe
    pre1961_df = pd.DataFrame({
        'year': pre1961_years,
        'sawnwood_domestic_inflow': pre1961_inflows['domestic']['sawnwood'],
        'woodpanels_domestic_inflow': pre1961_inflows['domestic']['woodpanels'],
        'paper_domestic_inflow': pre1961_inflows['domestic']['paper'],
        'sawnwood_export_inflow': pre1961_inflows['export']['sawnwood'],
        'woodpanels_export_inflow': pre1961_inflows['export']['woodpanels'],
        'paper_export_inflow': pre1961_inflows['export']['paper']
    })

    # Combine pre-1961 estimates with post-1961 data
    df = pd.concat([pre1961_df, df], ignore_index=True)
    df = df.sort_values('year').reset_index(drop=True)

    # Updated First-order decay function with initial stock and inflow timing options
    def fod_function(inflow, k, initial_stock_method='zero', inflow_timing='current', start_year=initial_year):
        stock = np.zeros(len(inflow))
        stock_change = np.zeros(len(inflow))
        
        # Set initial stock based on chosen method
        if initial_stock_method == 'zero':
            stock[0] = 0
        else:  # formula
            stock[0] = np.mean(inflow[0:5]) / k
            
        stock_change[0] = 0
        
        for i in range(1, len(inflow)):
            # Choose inflow value based on timing preference
            current_inflow = inflow[i-1] if inflow_timing == 'previous' else inflow[i]
            
            stock[i] = (stock[i-1] * np.exp(-k)) + (current_inflow * (1 - np.exp(-k)) / k)
            stock_change[i] = stock[i] - stock[i-1]
        
        return stock, stock_change

    # Calculate stocks and stock changes for domestic consumption
    sawnwood_domestic_stock, sawnwood_domestic_stock_change = fod_function(
        df['sawnwood_domestic_inflow'], k_sawnwood, initial_stock_method, inflow_timing)
    woodpanels_domestic_stock, woodpanels_domestic_stock_change = fod_function(
        df['woodpanels_domestic_inflow'], k_woodpanels, initial_stock_method, inflow_timing)
    paper_domestic_stock, paper_domestic_stock_change = fod_function(
        df['paper_domestic_inflow'], k_paper, initial_stock_method, inflow_timing)

    # Calculate stocks and stock changes for exports
    sawnwood_export_stock, sawnwood_export_stock_change = fod_function(
        df['sawnwood_export_inflow'], k_sawnwood, initial_stock_method, inflow_timing)
    woodpanels_export_stock, woodpanels_export_stock_change = fod_function(
        df['woodpanels_export_inflow'], k_woodpanels, initial_stock_method, inflow_timing)
    paper_export_stock, paper_export_stock_change = fod_function(
        df['paper_export_inflow'], k_paper, initial_stock_method, inflow_timing)

    # Add results to dataframe
    # Domestic stocks and changes
    df['sawnwood_domestic_stock'] = sawnwood_domestic_stock
    df['woodpanel_domestic_stock'] = woodpanels_domestic_stock
    df['paper_domestic_stock'] = paper_domestic_stock
    df['sawnwood_domestic_stock_change'] = sawnwood_domestic_stock_change
    df['woodpanels_domestic_stock_change'] = woodpanels_domestic_stock_change
    df['paper_domestic_stock_change'] = paper_domestic_stock_change
    
    # Export stocks and changes
    df['sawnwood_export_stock'] = sawnwood_export_stock
    df['woodpanel_export_stock'] = woodpanels_export_stock
    df['paper_export_stock'] = paper_export_stock
    df['sawnwood_export_stock_change'] = sawnwood_export_stock_change
    df['woodpanels_export_stock_change'] = woodpanels_export_stock_change
    df['paper_export_stock_change'] = paper_export_stock_change

    # Calculate CO2 removals
    df['domestic_stock_change'] = (sawnwood_domestic_stock_change + 
                                 woodpanels_domestic_stock_change + 
                                 paper_domestic_stock_change)
    df['export_stock_change'] = (sawnwood_export_stock_change + 
                                woodpanels_export_stock_change + 
                                paper_export_stock_change)
    df['total_stock_change'] = df['domestic_stock_change'] + df['export_stock_change']

    df['domestic_co2_removals'] = df['domestic_stock_change'] * (-44/12)
    df['export_co2_removals'] = df['export_stock_change'] * (-44/12)
    df['co2_removals'] = df['total_stock_change'] * (-44/12)

    # Filter and organize results
    results = df.copy()
    results = results[[
        'year',
        'total_stock_change',
        'co2_removals',
        
        # Domestic consumption columns
        'sawnwood_domestic_inflow',
        'woodpanels_domestic_inflow',
        'paper_domestic_inflow',
        'sawnwood_domestic_stock',
        'woodpanel_domestic_stock',
        'paper_domestic_stock',
        'sawnwood_domestic_stock_change',
        'woodpanels_domestic_stock_change',
        'paper_domestic_stock_change',
        'domestic_stock_change',
        'domestic_co2_removals',
        
        # Export columns
        'sawnwood_export_inflow',
        'woodpanels_export_inflow',
        'paper_export_inflow',
        'sawnwood_export_stock',
        'woodpanel_export_stock',
        'paper_export_stock',
        'sawnwood_export_stock_change',
        'woodpanels_export_stock_change',
        'paper_export_stock_change',
        'export_stock_change',
        'export_co2_removals'
    ]]

    # Export results to CSV
    output_file = 'Austria_HWP_carbon_estimates_results.csv'
    results.to_csv(output_file, index=False)
    print(f"\nResults exported to {output_file}")

except Exception as e:
    print(f"An error occurred: {e}")

# Additional check for filtering issues
print("\nYear range in data:")
print(df['year'].min(), df['year'].max())


Data loaded. Shape: (63, 17)

First few rows:
      Area  year  industrial_roundwood_export  industrial_roundwood_import  \
0  Austria  1961                     384100.0                     586400.0   
1  Austria  1962                     336400.0                     532900.0   
2  Austria  1963                     348400.0                     876700.0   
3  Austria  1964                     320100.0                     898400.0   
4  Austria  1965                     272200.0                     946800.0   

   industrial_roundwood_production  paper_export  paper_import  \
0                       10151000.0      205000.0        5700.0   
1                        9823000.0      197800.0        6700.0   
2                        8996000.0      210800.0        7200.0   
3                        9719000.0      231500.0        7700.0   
4                        9362000.0      254500.0        9500.0   

   paper_production  sawnwood_export  sawnwood_import  sawnwood_production  \
0         


Results exported to Austria_HWP_carbon_estimates_results.csv

Results from 1990 onwards:
 year  total_stock_change  co2_removals  sawnwood_domestic_inflow  woodpanels_domestic_inflow  paper_domestic_inflow  sawnwood_domestic_stock  woodpanel_domestic_stock  paper_domestic_stock  sawnwood_domestic_stock_change  woodpanels_domestic_stock_change  paper_domestic_stock_change  domestic_stock_change  domestic_co2_removals  sawnwood_export_inflow  woodpanels_export_inflow  paper_export_inflow  sawnwood_export_stock  woodpanel_export_stock  paper_export_stock  sawnwood_export_stock_change  woodpanels_export_stock_change  paper_export_stock_change  export_stock_change  export_co2_removals
 1990         1307872.625  -4795532.958               1285996.399                  352463.647             656706.442             38897773.189               4989353.500           1674280.394                      520796.881                        217125.484                    91364.685             829287.050   